In [7]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0)}")

CUDA available: True
CUDA device: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [39]:
import torch
import torch.nn as nn

# x = torch.randn(1, 5, 2)
x = torch.from_numpy(np.array([[[1,2], [3,4], [5,6], [7, 8], [9, 10]]])).float()
print(x.shape)
print(x)

conv1d = nn.Conv1d(in_channels=5, out_channels=3, kernel_size=2)

output = conv1d(x)
print(output.shape)
print(output)

torch.Size([1, 5, 2])
torch.Size([1, 3, 1])


In [40]:
print(x)

tensor([[[ 1.,  2.],
         [ 3.,  4.],
         [ 5.,  6.],
         [ 7.,  8.],
         [ 9., 10.]]])


In [41]:
print(output)

tensor([[[ 4.6974],
         [-7.4854],
         [-1.0097]]], grad_fn=<ConvolutionBackward0>)


In [39]:
from scripts.parsers import parse_sequences as parse_sequence_info

file_path = 'gait3d\\ListOfSequences.txt'
sequences = parse_sequence_info(file_path)

mocap_keys = []
par_cam_keys = []
par_cam_person = set()
par_after_cloth_change_keys = []
par_after_cloth_change_person = set()

for key, params in sequences.items():
    if params['MoCap_data']:
        mocap_keys.append(key)
        if key[-1] in ["1", "3", "5", "7"]:
            par_cam_keys.append(key)
            par_cam_person.add(key[:-2])
        if key[-1] in ["5", "7"]:
            par_after_cloth_change_keys.append(key)
            par_after_cloth_change_person.add(key[:-2])

print(f"Number of sequences with mocap data: {len(mocap_keys)}")
print(f"Number of sequences with mocap data and parallel cameras: {len(par_cam_keys)}")
print(f"Number of sequences with mocap data, parallel cameras and after clothing change: {len(par_after_cloth_change_keys)}")
print(f"Number of unique participants with mocap data and parallel cameras: {len(par_cam_person)}")
print(f"Number of unique participants with mocap data, parallel cameras and after clothing change: {len(par_after_cloth_change_person)}")
par_after_cloth_change_person

Number of sequences with mocap data: 146
Number of sequences with mocap data and parallel cameras: 72
Number of sequences with mocap data, parallel cameras and after clothing change: 12
Number of unique participants with mocap data and parallel cameras: 30
Number of unique participants with mocap data, parallel cameras and after clothing change: 6


{'p26', 'p27', 'p28', 'p29', 'p30', 'p31'}

In [40]:
import random

without_clothing_change = []
while len(without_clothing_change) < 6:
    random_person = random.choice(list(par_cam_person))
    if random_person not in par_after_cloth_change_person:
        without_clothing_change.append(random_person)
        par_cam_person.remove(random_person)

with_clothing_change = []
while len(with_clothing_change) < 4:
    random_person = random.choice(list(par_after_cloth_change_person))
    with_clothing_change.append(random_person)
    par_after_cloth_change_person.remove(random_person)


test_seq_set = ([f'{p_seq}s{seq_idx}' for p_seq in without_clothing_change[:3] for seq_idx in [1, 3]] +
                [f'{p_seq}s{seq_idx}' for p_seq in with_clothing_change[:2] for seq_idx in [5, 7]])

valid_seq_set = ([f'{p_seq}s{seq_idx}' for p_seq in without_clothing_change[3:] for seq_idx in [1, 3]] +
                [f'{p_seq}s{seq_idx}' for p_seq in with_clothing_change[2:] for seq_idx in [5, 7]])

print(f"test sequences: {test_seq_set}")
print(f"valid sequences: {valid_seq_set}")
# without_clothing_change + [only_after_clothing_change] + [only_before_clothing_change]

test sequences: ['p3s1', 'p3s3', 'p24s1', 'p24s3', 'p20s1', 'p20s3', 'p31s5', 'p31s7', 'p30s5', 'p30s7']
valid sequences: ['p23s1', 'p23s3', 'p10s1', 'p10s3', 'p8s1', 'p8s3', 'p29s5', 'p29s7', 'p27s5', 'p27s7']


In [41]:
train_seq_set = ([f'{p_seq}s{seq_idx}' for p_seq in list(par_cam_person) for seq_idx in [1, 3]] +
                 [f'{p_seq}s{seq_idx}' for p_seq in list(par_after_cloth_change_person) for seq_idx in [5, 7]])

print(f"train sequences: {train_seq_set}")

train sequences: ['p2s1', 'p2s3', 'p4s1', 'p4s3', 'p30s1', 'p30s3', 'p1s1', 'p1s3', 'p25s1', 'p25s3', 'p11s1', 'p11s3', 'p27s1', 'p27s3', 'p31s1', 'p31s3', 'p12s1', 'p12s3', 'p13s1', 'p13s3', 'p14s1', 'p14s3', 'p17s1', 'p17s3', 'p19s1', 'p19s3', 'p26s1', 'p26s3', 'p29s1', 'p29s3', 'p15s1', 'p15s3', 'p7s1', 'p7s3', 'p9s1', 'p9s3', 'p21s1', 'p21s3', 'p32s1', 'p32s3', 'p16s1', 'p16s3', 'p28s1', 'p28s3', 'p5s1', 'p5s3', 'p22s1', 'p22s3', 'p26s5', 'p26s7', 'p28s5', 'p28s7']


In [42]:
for key, params in sequences.items():
    if params['MoCap_data']:
        if key[-1] in ["1", "3", "5", "7"]:
            print(f"{key} | {'train' if key in train_seq_set else '     '} | {'valid' if key in valid_seq_set else '     '} | {'test' if key in test_seq_set else '    '} |")

p1s1 | train |       |      |
p1s3 | train |       |      |
p2s1 | train |       |      |
p2s3 | train |       |      |
p3s1 |       |       | test |
p3s3 |       |       | test |
p4s1 | train |       |      |
p4s3 | train |       |      |
p5s1 | train |       |      |
p5s3 | train |       |      |
p7s1 | train |       |      |
p7s3 | train |       |      |
p8s1 |       | valid |      |
p8s3 |       | valid |      |
p9s1 | train |       |      |
p9s3 | train |       |      |
p10s1 |       | valid |      |
p10s3 |       | valid |      |
p11s1 | train |       |      |
p11s3 | train |       |      |
p12s1 | train |       |      |
p12s3 | train |       |      |
p13s1 | train |       |      |
p13s3 | train |       |      |
p14s1 | train |       |      |
p14s3 | train |       |      |
p15s1 | train |       |      |
p15s3 | train |       |      |
p16s1 | train |       |      |
p16s3 | train |       |      |
p17s1 | train |       |      |
p17s3 | train |       |      |
p19s1 | train |       | 

In [49]:
print(f"Train size: {len(train_seq_set)} | {100*len(train_seq_set)/72:.2f}%")
print(f"Test size: {len(test_seq_set)} | {100*len(test_seq_set)/72:.2f}%")
print(f"Valid size: {len(valid_seq_set)} | {100*len(valid_seq_set)/72:.2f}%")

Train size: 52 | 72.22%
Test size: 10 | 13.89%
Valid size: 10 | 13.89%


In [ ]:
import json

selected_names_file = "./datasets/mediapipe/selected_joint_names.json"
input_data_file = "./datasets/mediapipe/dataset.json"
output_data = "./datasets/mocap/dataset.json"

with open(input_data, 'r') as file:
    raw_input = json.load(file)



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        # shape (12, 2) -> reshape to (2, 12) for Conv1d
        self.conv1d = nn.Conv1d(in_channels=2, out_channels=1, kernel_size=3)  # (2, 12) -> (1, 10)
        # Flatten 4 x 10 -> 40 x 1
        self.fc1 = nn.Linear(40, 64)
        self.fc2 = nn.Linear(64, 72)
        self.fc3 = nn.Linear(72, 36)

    def forward(self, x):
        # x: 4 tensors of shape (batch, 12, 2)
        conv_outs = []
        for xi in x:
            xi = xi.permute(0, 2, 1)  # reshape to (batch, 2, 12) for Conv1d
            conv = self.conv1d(xi)     # -> (batch, 1, 10)
            conv = conv.squeeze(1)     # -> (batch, 10)
            conv_outs.append(conv)

        concat = torch.cat(conv_outs, dim=1)  # -> (batch, 40)

        out = F.relu(self.fc1(concat))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)  # -> (batch, 36)
        out = out.view(-1, 12, 3)  # reshape to (batch, 12, 3)
        return out
